In [5]:
!pip install -q langchain openai chromadb
!pip install -q langchain-community
!pip install -q tiktoken

In [6]:
import os
os.environ["OPENAI_API_KEY"] = "sk-xxx"

# Download Sample Data

In [14]:
!pip install -q wget==3.2

In [15]:
# prompt: download this file from https://github.com/satriavid/StayChat-LLM-RAG-Chatbot/blob/main/data/uk_final_df.csv and put into /content

import os
import wget

url = "https://github.com/satriavid/StayChat-LLM-RAG-Chatbot/blob/main/data/uk_final_df.csv"
filename = os.path.basename(url)

wget.download(url, out=filename)

# Move the downloaded file to the desired location
#!mv {filename} /content


'uk_final_df.csv'

# Prepare Deployment
Streamlit as Frontend and NGROK as tunnel for accessing from public URL

In [7]:
!pip install -q streamlit pyngrok

In [8]:
with open('ai.py', 'w') as f:
    f.write("""
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain_community.embeddings.openai import OpenAIEmbeddings

loader = CSVLoader(file_path='/content/uk_final_df.csv')

# Create an index using the loaded documents
embedding = OpenAIEmbeddings()
index_creator = VectorstoreIndexCreator(embedding=embedding)
docsearch = index_creator.from_loaders([loader])

# Create a question-answering chain using the index
chain = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.vectorstore.as_retriever(), input_key="question")

def get_ai_response(query):
    response = chain({"question": query})
    return response['result']

""")

In [9]:
with open('app.py', 'w') as f:
    f.write("""
import streamlit as st
import ai

st.title("StayChat by NLP-B Ilya")

# Initialize chat history in session state
if 'messages' not in st.session_state:
    st.session_state['messages'] = [{"role": "assistant", "content": "Hello, How can I help you Today?"}]

# Display chat messages from history
for message in st.session_state['messages']:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Accept user input and display it
if prompt := st.chat_input("You:"):
    st.session_state['messages'].append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)

    # Generate and display assistant's response
    response = ai.get_ai_response(prompt)
    st.session_state['messages'].append({"role": "assistant", "content": response})
    with st.chat_message("assistant"):
        st.markdown(response)
""")

In [16]:
# @title Streamlit Server
from pyngrok import ngrok
import subprocess

# Set up ngrok with your authtoken
authtoken = ""
ngrok.set_auth_token(authtoken)

# Start the Streamlit server
process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Create an ngrok tunnel to the Streamlit server
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")

Streamlit app is live at: NgrokTunnel: "https://eea7-35-231-23-41.ngrok-free.app" -> "http://localhost:8501"


In [17]:
# @title Kill Streamlit and NGROK

import os
import signal
from pyngrok import ngrok

# Kill the Streamlit server process
if 'process' in globals():
    os.kill(process.pid, signal.SIGTERM)

# Disconnect ngrok
ngrok.kill()